In [1]:
# importing the required modules.
import random
import json
import numpy as np
import nltk
  
from keras.models import Sequential
from nltk.stem import WordNetLemmatizer
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

In [2]:
lemmatizer = WordNetLemmatizer()
  
# reading the json.intense file
intents = json.loads(open("intense.json").read())
  
# creating empty lists to store data
words = []
classes = []
documents = []
ignore_letters = ["?", "!", ".", ","]
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # separating words from patterns
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)  # and adding them to words list
          
        # associating patterns with respective tags
        documents.append(((word_list), intent['tag']))
  
        # appending the tags to the class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
  
# storing the root words or lemma
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))
  
# saving the words and classes list to json
json.dump(words, open('words.json', 'w'))
json.dump(classes, open('classes.json', 'w'))

In [3]:
# we need numerical values of the
# words because a neural network
# needs numerical values to work with
training = []
output_empty = [0]*len(classes)
for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
          
    # making a copy of the output_empty
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])
random.shuffle(training)
training = np.array(training)
  
# splitting the data
train_x = list(training[:, 0])
train_y = list(training[:, 1])


C:\Users\aadit\AppData\Local\Temp\ipykernel_10820\2443863883.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [4]:
# creating a Sequential machine learning model
model = Sequential()
model.add(Dense(32, input_shape=(len(train_x[0]), ), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
  

In [5]:
# compiling the model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd, metrics=['accuracy'])
hist = model.fit(np.array(train_x), np.array(train_y),
                 epochs=200, batch_size=5, verbose=1)

c:\Users\aadit\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/200
2/2 [==============================] - 0s 8ms/step - loss: 1.2081 - accuracy: 0.2222
Epoch 2/200
2/2 [==============================] - 0s 4ms/step - loss: 1.0452 - accuracy: 0.4444
Epoch 3/200
2/2 [==============================] - 0s 4ms/step - loss: 1.0650 - accuracy: 0.4444
Epoch 4/200
2/2 [==============================] - 0s 3ms/step - loss: 1.1246 - accuracy: 0.3333
Epoch 5/200
2/2 [==============================] - 0s 3ms/step - loss: 1.0245 - accuracy: 0.6667
Epoch 6/200
2/2 [==============================] - 0s 3ms/step - loss: 0.9748 - accuracy: 0.5556
Epoch 7/200
2/2 [==============================] - 0s 3ms/step - loss: 0.9833 - accuracy: 0.6667
Epoch 8/200
2/2 [==============================] - 0s 4ms/step - loss: 1.1411 - accuracy: 0.2222
Epoch 9/200
2/2 [==============================] - 0s 3ms/step - loss: 1.1045 - accuracy: 0.5556
Epoch 10/200
2/2 [==============================] - 0s 3ms/step - loss: 1.0239 - accuracy: 0.2222
Epoch 11/200
2/2 [===========

In [6]:
# saving the model
model.save("chatbotmodel", hist)
# print statement to show the 
# successful training of the Chatbot model
print("Yay!")

INFO:tensorflow:Assets written to: chatbotmodel\assets
Yay!


In [7]:
with np.printoptions(threshold=np.inf):
    print(model.layers[4].weights)

[<tf.Variable 'dense_2/kernel:0' shape=(16, 3) dtype=float32, numpy=
array([[-0.31058085,  0.1971391 , -0.06494314],
       [-0.18869738, -0.9725962 , -0.35791996],
       [ 0.33824265, -0.7216318 ,  0.5583419 ],
       [-0.667142  ,  0.5099098 ,  0.08366575],
       [ 0.42490333,  0.6408191 , -0.05741648],
       [-0.7427673 ,  0.6701266 ,  0.47552887],
       [-0.27935657,  0.9631942 , -0.2845665 ],
       [-0.55408216,  0.65282506,  0.4168293 ],
       [ 0.94881934, -0.02263902, -0.5151914 ],
       [ 0.3893145 , -0.41301313,  0.9309485 ],
       [ 0.18923861,  0.46510267, -0.00488741],
       [-0.871415  , -0.00549701,  0.4607519 ],
       [ 0.6011448 ,  0.18268658, -0.62869024],
       [ 0.9261575 ,  0.06645263, -0.43115076],
       [-0.15951309, -0.27381662, -0.45599788],
       [ 0.92111874, -0.6053457 , -0.23167881]], dtype=float32)>, <tf.Variable 'dense_2/bias:0' shape=(3,) dtype=float32, numpy=array([ 0.28392118, -0.31610164,  0.03218094], dtype=float32)>]
